In [16]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import ta
import matplotlib.pyplot as plt
import vnstock as vn
from datetime import timedelta

In [17]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_SLOW_PERIOD = 26
MACD_FAST_PERIOD = 12
MACD_SIGNAL_PERIOD = 9
initial_investment = 160_000_000
# backup_amount_initial = 40_000_000


In [18]:
win_rate = 0.5521109556
loss_rate = 1 - win_rate
mean_profit = 0.3944941585
mean_loss = 0.1314483279
stop_loss = 0.08
# take_profit = 0.15

In [19]:
def kelly_criterion(p, q, profit, loss):
    b = (profit * 160_000_000)/(loss * 160_000_000)
    f = (b*p - q)/b
    return f

In [20]:
kelly_criterion(win_rate,loss_rate,mean_profit,mean_loss)

0.4028710626825264

In [21]:
companies_vn30 = [
    'SSI', 'BCM','VHM','VIC','VRE','BVH','POW','GAS','ACB','BID',
'CTG','HDB','MBB','SSB','SHB','STB','TCB','TPB','VCB','VIB','VPB','HPG',
'GVR','MSN','VNM','SAB','VJC','MWG','PLX','FPT']

In [22]:
companies = ['VCB']

In [23]:
def calculate_indicators(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)

    return df

In [24]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['Previous_MACD'] < df['Previous_Signal_Line']) &
        (df['MACD'] >= df['Signal_Line']) &
        (df['RSI'] > RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] < RSI_OVERBOUGHT) &
        (df['Previous_MACD'] > df['Previous_Signal_Line']) &
        (df['MACD'] <= df['Signal_Line']), 'Signal'] = -1

    return df

In [25]:
def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

In [26]:
def simulate_investment(ticker, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction):
    try:
        data = vn.stock_historical_data(ticker, '2021-01-01', '2024-01-02', resolution='1D', type='stock')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = macd_strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        portfolio_values = []

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        # f_star = 1
        f_star = kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
        last_buy_price = 0  # Track the last buy price

        for i in range(len(data)):
            current_date = data.index[i]

            # Update pending transactions (T+2 settlement)
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            # Avoid initiating trades in January 2024
            if current_date.month == 1 and current_date.year == 2024:
                current_value = cash + holdings * data['close'].iloc[i]
                portfolio_values.append(current_value)
                continue

            if current_date in buy_signals:
                # Invest 
                allocation = cash * f_star
                if allocation > 0:
                    shares_to_buy = int(allocation // data['close'][i])
                    if shares_to_buy > 0:
                        total_cost = shares_to_buy * data['close'][i]
                        if cash >= total_cost:
                            cash -= total_cost
                        else:
                            shares_to_buy = int(cash // data['close'][i])
                            total_cost = shares_to_buy * data['close'][i]
                            cash -= total_cost

                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    if settlement_date in pending_buy_shares:
                        pending_buy_shares[settlement_date] += shares_to_buy
                    else:
                        pending_buy_shares[settlement_date] = shares_to_buy

                    last_buy_price = data['close'].iloc[i]  # Track the last buy price


            # Check if we need to sell due to sell signal, stop-loss, or take-profit
            current_price = data['close'].iloc[i]
            if holdings > 0:
                should_sell = False

                # Check sell signal
                if current_date in sell_signals:
                    should_sell = True
                    reason = 'sell signal'

                # Check stop-loss
                # elif current_price <= last_buy_price * (1 - stop_loss):
                #     should_sell = True
                #     reason = 'stop loss'

                # Check take-profit
                # elif current_price >= last_buy_price * (1 + take_profit):
                #     should_sell = True

                if should_sell:
                    shares_to_sell = int(holdings * sell_fraction)
                    if shares_to_sell > 0:
                        if shares_to_sell > holdings:
                            shares_to_sell = holdings
                        revenue = shares_to_sell * current_price 
                        holdings -= shares_to_sell
                        settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                        if settlement_date in pending_sell_revenue:
                            pending_sell_revenue[settlement_date] += revenue
                        else:
                            pending_sell_revenue[settlement_date] = revenue

            current_value = cash + holdings * data['close'].iloc[i]
            portfolio_values.append(current_value)

        # Calculate final portfolio value including pending transactions
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)

            current_value = cash + holdings * data['close'].iloc[-1]
            portfolio_values.append(current_value)
            final_date += BDay(1)

        # Ensure the length of portfolio_values matches the length of data index
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

In [27]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / (initial_investment)  * 100
            })
    return pd.DataFrame(results)

In [30]:
sell_fraction = kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction)

# Save results to CSV
# results_df.to_csv('MACDt2.csv', index=False)

# Print results
# print(results_df)

average_rate_of_return = results_df['Rate of Return'].mean()
average_return = results_df[results_df['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = results_df[results_df['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(average_rate_of_return, average_return, average_loss)

C:\Users\Admin\AppData\Local\Temp\ipykernel_1220\245259429.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_1220\245259429.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

Average Rate of Return for 30 companies: 7.0303682499999995
7.0303682499999995 20.718793382352942 -10.869880000000002


In [29]:
data = vn.stock_historical_data('VCB', '2021-01-01', '2024-01-03', resolution='1D', type='stock')
data = data.set_index(pd.DatetimeIndex(data['time'].values))
data = calculate_indicators(data)
data = macd_strategy(data)
print(data)


                  time   open   high    low  close   volume ticker        RSI  \
2021-01-04  2021-01-04  64269  65450  63810  64930   855900    VCB        NaN   
2021-01-05  2021-01-05  64930  65980  64930  65120   739200    VCB        NaN   
2021-01-06  2021-01-06  65250  69650  65120  68860  2080400    VCB        NaN   
2021-01-07  2021-01-07  68200  71160  68200  70170   889000    VCB        NaN   
2021-01-08  2021-01-08  70830  70830  68530  68730   963900    VCB        NaN   
...                ...    ...    ...    ...    ...      ...    ...        ...   
2023-12-27  2023-12-27  82900  83200  82500  82700   899600    VCB  43.606020   
2023-12-28  2023-12-28  82700  83100  82400  82800   703200    VCB  44.328591   
2023-12-29  2023-12-29  83500  83900  80300  80300  1621700    VCB  32.958965   
2024-01-02  2024-01-02  82900  83600  82200  83500  1785800    VCB  50.470400   
2024-01-03  2024-01-03  83500  84500  82800  84500  1373000    VCB  54.472513   

            Previous_RSI   

C:\Users\Admin\AppData\Local\Temp\ipykernel_1220\245259429.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_1220\245259429.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe